# Load Modules

In [6]:
import os, sys
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate
import time
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.patches as patches
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap


import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from numpy import *
import numpy as np
from pylab import *
import time
from datetime import date, timedelta
from matplotlib import animation
import matplotlib.animation as animation
import types
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import nclcmaps
import pandas as pd
import xarray as xr
from scipy import interpolate
import operator
import multiprocessing
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.patches as patches

# Read in CM1 Output

In [7]:
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/steenburgh-group8/tom/cm1/output/15ms_1500m_tug.nc', chunks={'nk': 5})
#Get model output demensions
num_x = ds.nx
num_y = ds.ny
num_z = ds.nz

x = np.arange(0,num_x,1)
y = np.arange(0,num_y,1)
z = np.arange(0,num_z,1)

# Create Dask Cluster

In [8]:
#Option to use multiple processors and threads 
from dask.distributed import Client, LocalCluster
c = LocalCluster(n_workers=20, threads_per_worker=1)
client = Client(c)
client

distributed.scheduler - ERROR - Failed to connect to worker 'tcp://127.0.0.1:39747': can't start new thread
distributed.scheduler - ERROR - 'tcp://127.0.0.1:39747'
Traceback (most recent call last):
  File "/uufs/chpc.utah.edu/common/home/u1013082/anaconda3/lib/python3.6/site-packages/distributed/scheduler.py", line 2125, in handle_worker
    comm = yield connect(worker, connection_args=self.connection_args)
  File "/uufs/chpc.utah.edu/common/home/u1013082/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/uufs/chpc.utah.edu/common/home/u1013082/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/uufs/chpc.utah.edu/common/home/u1013082/anaconda3/lib/python3.6/site-packages/distributed/comm/core.py", line 187, in connect
    quiet_exceptions=EnvironmentError)
  File "/uufs/chpc.utah.edu/common/home/u1013082/anaconda3/lib/python3.6/site-packages/tornado/gen.py"

TimeoutError: Worker failed to start

# Info to Initialize and Calculate Parcels

Variables based on their model output and desired parcels (set by user). The code initializes an array of parcels in the the y,z domain. It can be easily modified to also initialize in the x-domain.

In [ ]:
num_seeds_z = 30 #Number of parcels in vertical (can be more than number of vertical levels) 
num_seeds_y = 1 #Number of parcels in y
time_steps = 5 #Number of time steps to run trajectories back
start_time_step = 210 #Time step to start backwards trajectories at
hor_resolution = 150 #Horizontal resolution of model output (meters)
vert_resolution = 100 #Vertical resolution of model output. Only used in no-terrain runs. Calculated below for runs with terrain (meters)
time_step_length = 90.0 #Model output time step length (seconds)
var_name1 = 'th' #Variable to record at each parcel's location throughout trajectory (code can be easily modified to add more)

Create empty arrays to x, y, and z positions of parcels and variable to track

In [ ]:
xpos = np.zeros((time_steps, num_seeds_z, num_seeds_y))
ypos = np.zeros((time_steps, num_seeds_z, num_seeds_y))
zpos = np.zeros((time_steps, num_seeds_z, num_seeds_y))
zpos_grid = np.zeros((time_steps, num_seeds_z, num_seeds_y))
change_zs = np.zeros((time_steps, num_seeds_z, num_seeds_y))
variable1 = np.zeros((time_steps, num_seeds_z, num_seeds_y))

Initial location of seeds (set by user)

In [ ]:
#x-position
xpos[0,:,:] = num_x-410 #This example initializes all seeds at same x-position

#y-position   
for i in range(num_seeds_y):
    ypos[0,:,i] = 340 #This example initializes seeds evenly starting at 0 in y-dimension

#z-position
for i in range(num_seeds_z):
    zpos[0,i,:] = 2*i #This example initializes seeds evenly starting at zero in z-deimension

## Initialize Surface Terrain Height and Calculate Vertical Grid-Spacing
CM1 output in terrain following so we must add the surface terrain height to the initial height of the parcels above the surface. Also, the vertical grid spacing changes above areas of terrain in CM1. We must read in the the vertical heights and calculate the the vertical grid spacing for each x,y grid point. This will be ignored if you are not using terrain.

In [ ]:
#check if run has terrain (if not, zs is zero and z-position should not be affected)
try:
    #Get vertical coordinates
    zh = np.array(ds.zh[0,:,:,:])
    #Create list of current coordinates for terrain addition
    xloc = np.array(xpos[0,:,:]).flatten()
    yloc = np.array(ypos[0,:,:]).flatten()
    zloc = np.array(zpos[0,:,:]).flatten()
    coor_terrain = []
    for i in range(len(xloc)):
        coor_terrain.append((zloc[i], yloc[i], xloc[i]))
    
    #Get the actual inital height of the parcels. Not their height above the surface.
    zpos_terrain = np.array(zpos)
    zpos_terrain[0,:,:] = np.reshape(interpolate.interpn((z,y,x), zh, coor_terrain, method='linear', bounds_error=False, fill_value= 0), (num_seeds_z, num_seeds_y))
    
    #This gets us the grid spacing for the vertical grid
    z_grid = zh[1:,:,:]-zh[:-1,:,:]


except:
    #No terrain runs
    zh = np.zeros((ds.nz,ds.ny, ds.nx))
    zpos_terrain = np.copy(zpos)*vert_resolution
    z_grid = np.ones((ds.nz-1,ds.ny, ds.nx))*vert_resolution

# Calculate Trajectories

In [ ]:
#Loop over all time steps and compute trajectory
for t in range(time_steps-1):
    
    start = time.time() #Timer
    
    #Get model data
    u = ds.uinterp[start_time_step-t,:,:,:].values
    v = ds.vinterp[start_time_step-t,:,:,:].values
    w = ds.winterp[start_time_step-t,:,:,:].values
    var1 = getattr(ds,var_name1)[start_time_step-t,:,:,:].values

                                
    #####################   Get x,y,z for variables ###########################
    ##  If the model output is terrain following, the change in surface      ##
    ##  height at each timestep must be added to the height (zpos) of the    ##
    ##  parcels.                                                             ##
    ###########################################################################
        
    #Get surface height grid
    try:
        zs = np.array(ds.zs[0,:,:])
    except:
        zs = np.zeros((ds.ny, ds.nx))  


    #Get previous and current x and y positions to calc change in surface height
    if t > 0:
        
        #x and y for one time step back
        xloc = np.copy(xpos[t-1,:,:]).flatten()
        yloc = np.copy(ypos[t-1,:,:]).flatten()
        coor_xminus1 = []
        for i in range(len(xloc)):
            coor_xminus1.append((yloc[i], xloc[i]))
            
            
        #xand y for current time step
        xloc = np.copy(xpos[t,:,:]).flatten()
        yloc = np.copy(ypos[t,:,:]).flatten()
        coor_x = []
        for i in range(len(xloc)):
            coor_x.append((yloc[i], xloc[i]))
            
        #Calc surface height at each time
        xminus1_h = interpolate.interpn((y,x), zs, coor_xminus1, method='linear', bounds_error=False, fill_value= 0)
        x_h = interpolate.interpn((y,x), zs, coor_x, method='linear', bounds_error=False, fill_value= 0)
        
        #Calc change in surface height during last timestep, which will be added to zpos
        change_zs[t,:,:] =  np.reshape(x_h - xminus1_h, (num_seeds_z, num_seeds_y))
    
    
    #Get get x, y, and z positions to calc parcel movement
    xloc = np.copy(xpos[t,:,:]).flatten()
    yloc = np.copy(ypos[t,:,:]).flatten()
    zloc = np.copy(zpos[t,:,:]).flatten()
    coor_var = []
    for i in range(len(xloc)):
        coor_var.append((zloc[i], yloc[i], xloc[i])) 
    
    
    #####################   Calc new xpos #####################################
    xpos[t+1,:,:] = xpos[t,:,:] - np.reshape(interpolate.interpn((z,y,x), u, coor_var, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))

    #####################   Calc new ypos #####################################
    ypos[t+1,:,:]  = ypos[t,:,:] - np.reshape(interpolate.interpn((z,y,x), v, coor_var, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))

    #####################   Calc new zpos #####################################
    #zpos grid spacing
    zpos_grid[t,:,:] = np.reshape(interpolate.interpn((z[:-1],y,x), z_grid, coor_var, method='linear', bounds_error=False, fill_value= 0), (num_seeds_z, num_seeds_y))
    #terrain-following (includes change in surface height)
    zpos[t+1,:,:]  = zpos[t,:,:] - change_zs[t,:,:]/zpos_grid[t,:,:] - np.reshape(interpolate.interpn((z,y,x), w, coor_var, method='linear', bounds_error=False, fill_value= 0), (num_seeds_z, num_seeds_y))*time_step_length/zpos_grid[t,:,:]
    #terrain-height coordinates
    zpos_terrain[t+1,:,:]  = zpos_terrain[t,:,:] - np.reshape(interpolate.interpn((z,y,x), w, coor_var, method='linear', bounds_error=False, fill_value= 0)*time_step_length, (num_seeds_z, num_seeds_y))

    zpos = zpos.clip(min=0) #Prevents parcel from going into the ground
    
        
    #Prevents parcel from going into ground
    xloc = np.copy(xpos[t,:,:]).flatten()
    yloc = np.copy(ypos[t,:,:]).flatten()
    coor_terrain = []
    for i in range(len(xloc)):
        coor_terrain.append((yloc[i], xloc[i])) 
    surface_height = np.reshape(interpolate.interpn((y,x), zs, coor_terrain, method='linear', bounds_error=False, fill_value= 0)/vert_resolution, (num_seeds_z, num_seeds_y))
    zpos_terrain[t,:,:] = zpos_terrain[t,:,:].clip(min=surface_height) 
    
    
    #Variables
    variable1[t,:,:] = np.reshape(interpolate.interpn((z,y,x), var1, coor_var, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))  
    
    #Timer
    stop = time.time()
    print("Integration {:01d} took {:.2f} seconds".format(t, stop-start))
    
    
    
    
##############  Get variable data for final time step #########################
t = time_steps-1

#Get get x, y, and z positions to calc parcel movement
xloc = np.copy(xpos[t,:,:]).flatten()
yloc = np.copy(ypos[t,:,:]).flatten()
zloc = np.copy(zpos[t,:,:]).flatten()
coor_var = []
for i in range(len(xloc)):
    coor_var.append((zloc[i], yloc[i], xloc[i])) 


#Variables
variable1[t,:,:] = np.reshape(interpolate.interpn((z,y,x), var1, coor_var, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))

# Save Trajectory Data
The x, y, and z position and user-specified variable values are saved in 3D numpy arrays. The first dimension is time and the other two are the positions and values of variables of all the parcels at that specifc time.

In [ ]:
np.save('xpos', xpos)
np.save('ypos', ypos)
np.save('zpos', zpos_terrain)
np.save('%s' %var_name1, variable1)

# Plot

In [ ]:
print(zpos_terrain[0,:,:])
print(zpos[0,:,:])

In [ ]:
###############################################################################
###################  Backward Trajectory Plot  ################################
###############################################################################

fig = plt.figure(num=None, figsize=(12,8), facecolor='w', edgecolor='k')
%matplotlib inline

###################  Plot Trajectories in xy  ###############################

#Plot characteristics
ax = plt.subplot(211,aspect = 'equal')
#plt.title("Backward Trajectories", fontsize = 18, y = 1.05) 
plt.subplots_adjust(left=0.1, bottom=0.11, right=0.87, top=0.93, wspace=0, hspace=0.15)

xlmin = 262
xlmax = 278.01
xlevels = np.arange(xlmin,xlmax, 0.05)
xlevels_ticks = np.arange(xlmin,xlmax,2)
xlevels_ticks_labels = np.arange(xlmin,xlmax, 2).astype(int)

###############################################################################

import nclcmaps
#Read in colormap and put in proper format
colors1 = np.array(nclcmaps.colors['WhiteBlueGreenYellowRed'])#perc2_9lev'])
colors_int = colors1.astype(int)
colors = list(colors_int)
cmap_th = nclcmaps.make_cmap(colors, bit=True)


cmap = cmap_th
trajectories_xy = plt.scatter(xpos, ypos, c = variable1, cmap = cmap_th, norm=matplotlib.colors.BoundaryNorm(xlevels,cmap.N), s = 1, zorder = 7)



plt.xticks(np.arange(0,ds.uinterp[0,0,0,:].size,200))
plt.yticks(np.arange(0,ds.uinterp[0,0,:,0].size,100))
ax.set_xticklabels(np.arange(0,ds.uinterp[0,0,0,:].size*200/100,40), fontsize = 14)
ax.set_yticklabels(np.arange(0,ds.uinterp[0,0,0,:].size*200/100,40), fontsize = 14)
plt.ylabel('Distance (km)', fontsize = 20)
plt.xlim([0,ds.uinterp[0,0,0,:].size])
plt.ylim([0,ds.uinterp[0,0,:,0].size])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

#Water and land details
levels_water = [1.5, 2.5]
levels_terrain = [0,1.5]
xland_plt = plt.contourf(ds.xland[0,:,:], levels_water, alpha = 1, colors = ('lightsteelblue'), zorder = 2)
xland_plt = plt.contour(ds.xland[0,:,:], levels_water, alpha = 1, colors = ('k'), zorder = 10)
xland_plt = plt.contourf(ds.xland[0,:,:], levels_terrain, alpha = 1, colors = ('gainsboro'), zorder = 1)

#Terrain
terrain_levels = np.arange(-1, 3000.1, 200)
try:
    zs = ds.zs[0,:,:].values
except:
    zs = np.zeros((ds.ny,ds.nx))

terrain = plt.contourf(zs[:,:], terrain_levels, alpha = 1, cmap = cm.Greys, vmin = -800, vmax = 3000, zorder = 1)




###################  Plot Trajectories in xz  ###############################


#Plot characteristics
ax = plt.subplot(212)


trajectories_xz = plt.scatter(xpos, zpos_terrain, c = variable1, cmap = cmap_th, norm=matplotlib.colors.BoundaryNorm(xlevels,cmap.N), s = 1, zorder = 3)




plt.grid(True, color = 'white', )
plt.ylabel('Height (m)', fontsize = 20)
plt.xlabel('Distance (km)', fontsize = 20)
plt.xticks(np.arange(0,ds.nx,40*1000/200))
ytick = np.arange(0,4001,500)
plt.yticks(ytick)
ax.set_xticklabels(np.arange(0,ds.nx*200/1000,40), fontsize = 14)
plt.xlim([0,ds.uinterp[0,0,0,:].size])
plt.ylim([-50,ytick[-1]])
ax.tick_params(axis='y', labelsize=14)
plt.axvspan(0,ds.uinterp[0,0,0,:].size,color='gainsboro',lw=0)

#Labels
#sub_title = ['[Run: Low Wind]']
#props = dict(boxstyle='square', facecolor='white', alpha=1)
#ax.text(1800, -850, sub_title[0], fontsize = 15, zorder = 5)


#Plot Terrain
x1d = np.arange(0,ds.nx,1)
ymid = np.int(ds.ny/2)
terrain = plt.plot(x1d, zs[ymid,:]-40, c = 'slategrey', linewidth = 4, zorder = 4)

#Plot Lake
lake = np.array(ds.xland[0,ymid,:])
lake[lake == 1] = np.nan
lake_plt = plt.plot(x1d, lake-40, c = 'blue', linewidth = 4, zorder = 5)

#Colorbar
cbaxes = fig.add_axes([0.9, 0.27, 0.035, 0.52])             
cbar = plt.colorbar(trajectories_xz, cax = cbaxes, ticks = xlevels_ticks)
cbar.ax.set_yticklabels(xlevels_ticks_labels)
cbar.ax.tick_params(labelsize=14)
plt.text(-0.6, -0.1, 'Theta (K)', fontsize = 20)



plt.savefig("backward_trajectory.png", dpi = 250)
plt.close(fig)